# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [9]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [10]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [11]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [12]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

In [13]:
# Some imports for handling images

import base64

# Image Library
from io import BytesIO
from PIL import Image

def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [28]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [47]:
# There's a particular dictionary structure that's required to describe our function:

destinations = ["london", "paris", "tokyo", "berlin"]

def get_destination():
    possible_dest = ""
    
    for dst in destinations:
        possible_dest += dst
        possible_dest += ", "

    return possible_dest

destination_function = {
    "name": "get_destination",
    "description": "get the possible destination for airline guests. Call thus whenever you need to know all possible destinations of out airline, for example when a customer asks 'What is possible destinations of this airline?'",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
        "additionalProperties": False
    
    }
}

# And this is included in a list of tools:

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": destination_function} ]

Tool get_ticket_price called for 도쿄


Tool get_ticket_price called for 파리


In [44]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None

    message = response.choices[0].message
    
    if response.choices[0].finish_reason=="tool_calls":
        if message.tool_calls[0].function.name == 'get_ticket_price':
            response, city = handle_tool_call(message)
            image = artist(city)
        else:
            response = handle_tool_call(message)

        messages.append(message)
        messages.append(response)

        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    reply = translator(reply).choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image


# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == 'get_ticket_price':
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id
        }
        return response, city
    elif tool_call.function.name == 'get_destination':
        destinations = get_destination()
        response = {
            "role": "tool",
            "content": destinations,
            "tool_call_id": tool_call.id
        }
        return response
    else: 
        print("Tool call fault")
        return 0

In [52]:
def translator(reply):
    sys_prom = "당신은 한국어 번역가입니다. 사용자의 입력을 한국어로만 번역해 출력하세요. \
                추가 설명 금지. 코드/마크다운/URL은 보존. \
                사용자가 한국어를 포함한 어떠한 언어로 질문하던지와 관계없이 모든 대답은 반드시 한국어로 번역해 출력하세요."
    
    messages = [
        {"role": "system", "content": sys_prom},
        {"role": "user", "content": reply}
    ]
    
    response=openai.chat.completions.create(model = MODEL, messages=messages)

    return response

In [53]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    # 첫 번째 행
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    # 두 번째 행
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    # 세 번째 행
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.
